<a href="https://colab.research.google.com/github/rohandiaz/Hack_Heritage_ML/blob/main/Recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install scikit-surprise


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 1.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357284 sha256=65d2a77ba92eac53fe7435c3fa99fa3b4e5496f377901871a0d0290887dc0deb
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


In [ ]:
import pandas as pd
import numpy as np
from surprise import Dataset, Reader, SVD
from surprise import Reader, Dataset, KNNBasic
from surprise.model_selection import train_test_split
from surprise import accuracy


In [ ]:
print("Hi this is Rohan here")

Hi this is Rohan here


In [ ]:
# Create dummy clients data
clients = pd.DataFrame({
    'client_id': [1, 2, 3, 4, 5],
    'client_name': ['Alice', 'Bob', 'Charlie', 'David', 'Eve'],
    'industry': ['Web Development', 'Graphic Design', 'Content Writing', 'Data Science', 'Marketing'],
    'budget_range': [1000, 500, 700, 1500, 1200]
})

# Create dummy freelancers data
freelancers = pd.DataFrame({
    'freelancer_id': [101, 102, 103, 104, 105],
    'freelancer_name': ['John', 'Emma', 'Liam', 'Sophia', 'Oliver'],
    'skills': ['Python, Django', 'Photoshop, Illustrator', 'Content Writing', 'Python, ML', 'SEO, Marketing'],
    'hourly_rate': [50, 40, 30, 70, 60],
    'experience_level': ['Intermediate', 'Senior', 'Junior', 'Senior', 'Intermediate']
})

# Create dummy interaction data (Client-Freelancer interactions, e.g., past projects)
interactions = pd.DataFrame({
    'client_id': [1, 1, 2, 3, 4, 5],
    'freelancer_id': [101, 102, 102, 103, 104, 105],
    'project_rating': [5, 4, 4, 3, 5, 4],
    'project_budget': [1000, 500, 500, 700, 1500, 1200]
})


In [ ]:
# Merge client data with interaction data
client_freelancer_data = pd.merge(interactions, clients, on='client_id')
client_freelancer_data = pd.merge(client_freelancer_data, freelancers, on='freelancer_id')

# Display the merged data
client_freelancer_data.head()


,client_id,freelancer_id,project_rating,project_budget,client_name,industry,budget_range,freelancer_name,skills,hourly_rate,experience_level
0,1,101,5,1000,Alice,Web Development,1000,John,"Python, Django",50,Intermediate
1,1,102,4,500,Alice,Web Development,1000,Emma,"Photoshop, Illustrator",40,Senior
2,2,102,4,500,Bob,Graphic Design,500,Emma,"Photoshop, Illustrator",40,Senior
3,3,103,3,700,Charlie,Content Writing,700,Liam,Content Writing,30,Junior
4,4,104,5,1500,David,Data Science,1500,Sophia,"Python, ML",70,Senior


In [ ]:
# Create a pivot table (interaction matrix) with clients as rows and freelancers as columns
interaction_matrix = client_freelancer_data.pivot_table(index='client_name', columns='freelancer_name', values='project_rating').fillna(0)

# Display the interaction matrix
interaction_matrix


freelancer_name,Emma,John,Liam,Oliver,Sophia
client_name,,,,,
Alice,4.0,5.0,0.0,0.0,0.0
Bob,4.0,0.0,0.0,0.0,0.0
Charlie,0.0,0.0,3.0,0.0,0.0
David,0.0,0.0,0.0,0.0,5.0
Eve,0.0,0.0,0.0,4.0,0.0


In [ ]:
# Let's say we want to recommend freelancers to client with id 1
client_id = 1

# Get a list of all freelancer_ids
freelancer_ids = freelancers['freelancer_id'].unique()

# List freelancers the client has already rated
freelancers_rated_by_client = interactions[interactions['client_id'] == client_id]['freelancer_id'].unique()

# Find freelancers the client hasn't rated yet
freelancers_not_rated = [fid for fid in freelancer_ids if fid not in freelancers_rated_by_client]

# Predict ratings for freelancers not rated by the client
recommendations = []
for freelancer_id in freelancers_not_rated:
    pred = algo.predict(client_id, freelancer_id)
    recommendations.append((freelancer_id, pred.est))

# Sort the recommendations by the predicted rating in descending order
recommendations.sort(key=lambda x: x[1], reverse=True)

# Print the top recommendations
top_n = 3  # Number of top recommendations to display
print("Top recommended freelancers for client {}: ".format(client_id))
for freelancer_id, predicted_rating in recommendations[:top_n]:
    print("Freelancer ID: {}, Predicted Rating: {:.2f}".format(freelancer_id, predicted_rating))


Top recommended freelancers for client 1: 
Freelancer ID: 103, Predicted Rating: 4.50
Freelancer ID: 104, Predicted Rating: 4.50
Freelancer ID: 105, Predicted Rating: 4.50


In [ ]:
# Prepare data for Surprise
# We're using the 'interactions' dataframe created earlier, but it needs to be in (client_id, freelancer_id, project_rating) format

# Only select the necessary columns
data = interactions[['client_id', 'freelancer_id', 'project_rating']]

# Define the format of the data (rating scale should be within 1-5)
reader = Reader(rating_scale=(1, 5))

# Load the data into Surprise's Dataset
dataset = Dataset.load_from_df(data, reader)


In [ ]:
# Split the dataset into training and test sets (80% train, 20% test)
trainset, testset = train_test_split(dataset, test_size=0.2)


In [ ]:
# Use KNNBasic algorithm for user-based collaborative filtering
# 'user-based' is set to True
sim_options = {
    'name': 'cosine',
    'user_based': True  # User-Based CF
}

# Define the model
algo = KNNBasic(sim_options=sim_options)

# Train the model on the training set
algo.fit(trainset)


Computing the cosine similarity matrix...
Done computing similarity matrix.


In [ ]:
# Predict ratings for the testset
predictions = algo.test(testset)

# Evaluate the model's performance (Root Mean Squared Error)
accuracy.rmse(predictions)


RMSE: 0.7071


0.7071067811865476

In [ ]:
from surprise import SVD

# Initialize SVD model
algo = SVD()

# Train the model
algo.fit(trainset)

# Predict ratings
predictions = algo.test(testset)

# Evaluate the model's performance using RMSE
accuracy.rmse(predictions)


RMSE: 0.7170


0.7169970878115598

In [ ]:
sim_options = {
    'name': 'pearson',   # Change to Pearson similarity
    'user_based': True   # User-based Collaborative Filtering
}

algo = KNNBasic(k=10, sim_options=sim_options)
algo.fit(trainset)

# Test and evaluate
predictions = algo.test(testset)
accuracy.rmse(predictions)


Computing the pearson similarity matrix...
Done computing similarity matrix.
RMSE: 0.7071


0.7071067811865476

In [ ]:
print(interaction_matrix.describe())


freelancer_name     Emma      John      Liam    Oliver    Sophia
count            5.00000  5.000000  5.000000  5.000000  5.000000
mean             1.60000  1.000000  0.600000  0.800000  1.000000
std              2.19089  2.236068  1.341641  1.788854  2.236068
min              0.00000  0.000000  0.000000  0.000000  0.000000
25%              0.00000  0.000000  0.000000  0.000000  0.000000
50%              0.00000  0.000000  0.000000  0.000000  0.000000
75%              4.00000  0.000000  0.000000  0.000000  0.000000
max              4.00000  5.000000  3.000000  4.000000  5.000000


In [ ]:
# Load a sample dataset (movie ratings dataset)
data = Dataset.load_builtin('ml-100k')  # MovieLens 100k dataset

# Split the dataset into training and testing sets
trainset, testset = train_test_split(data, test_size=0.2)

# Use Singular Value Decomposition (SVD) for Matrix Factorization
model = SVD()

# Train the model
model.fit(trainset)

# Test the model on the test set
predictions = model.test(testset)

# Calculate and print the accuracy (RMSE)
accuracy.rmse(predictions)

# Make predictions for a specific user and item
user_id = '196'  # Example user ID
item_id = '242'  # Example item ID

prediction = model.predict(user_id, item_id)
print(f"Predicted rating for User {user_id} and Item {item_id}: {prediction.est}")

Dataset ml-100k could not be found. Do you want to download it? [Y/n] y
Trying to download dataset from https://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k
RMSE: 0.9380
Predicted rating for User 196 and Item 242: 3.8718824808108616


In [ ]:


# Load the MovieLens 100k dataset
data = Dataset.load_builtin('ml-100k')  # You can replace this with your custom dataset

# Split the dataset into training and testing sets
trainset, testset = train_test_split(data, test_size=0.2)

# Use Singular Value Decomposition (SVD) for Matrix Factorization
model = SVD()

# Train the model
model.fit(trainset)

# Test the model on the test set
predictions = model.test(testset)

# Calculate and print the accuracy (RMSE)
accuracy.rmse(predictions)

# Build a ranking system

# Get a list of all item IDs (items are the 'movie' in the dataset)
all_items = set([str(i) for i in range(1, 1683)])  # MovieLens 100k has 1682 items

# Get the list of all users
all_users = set([str(i) for i in range(1, 944)])  # MovieLens 100k has 943 users

# For example, let’s create rankings for a specific user
user_id = '196'

# Get the items the user has already interacted with (rated)
user_rated_items = [trainset.to_raw_iid(iid) for iid, rating in trainset.ur[trainset.to_inner_uid(user_id)]]

# Get items the user hasn't interacted with
unrated_items = all_items - set(user_rated_items)

# Predict ratings for all unrated items
predictions = [model.predict(user_id, item_id) for item_id in unrated_items]

# Sort the predictions by estimated rating in descending order
predictions.sort(key=lambda x: x.est, reverse=True)

# Display top 10 recommended items with their predicted ratings
top_n = 10
top_recommendations = predictions[:top_n]

# Print top recommendations
print(f"Top {top_n} recommendations for User {user_id}:")
for prediction in top_recommendations:
    print(f"Item {prediction.iid} - Predicted rating: {prediction.est:.2f}")



RMSE: 0.9408
Top 10 recommendations for User 196:
Item 318 - Predicted rating: 4.62
Item 169 - Predicted rating: 4.61
Item 174 - Predicted rating: 4.60
Item 114 - Predicted rating: 4.58
Item 64 - Predicted rating: 4.55
Item 408 - Predicted rating: 4.52
Item 12 - Predicted rating: 4.48
Item 483 - Predicted rating: 4.47
Item 187 - Predicted rating: 4.46
Item 513 - Predicted rating: 4.45
